In [84]:
import Estimate_Covariance_Matrix as ECM
from portfolio_choice_functions import ew_implement
from prepare_portfolio_data import define_important_dates, create_date_ranges
from Main import settings, features, pf_set
import pandas as pd
from datetime import datetime
from pandas.tseries.offsets import MonthEnd
from Prepare_Data import process_risk_free_rate, load_and_filter_market_returns_test, wealth_func
import os
import pickle
import time
import prepare_portfolio_data
import data_run_files
import Estimate_Covariance_Matrix
import Prepare_Data
import portfolio_choice_functions
from portfolio_choice_functions import w_fun, m_func
import portfolio_choice_functions
import numpy as np
from dateutil.relativedelta import relativedelta
import importlib
import General_Functions
sqrtm_cpp = importlib.import_module("sqrtm_cpp")
import time
from datetime import timedelta
from return_prediction_functions import rff
import General_Functions
from pandas.tseries.offsets import MonthEnd

import matplotlib.pyplot as plt
import seaborn as sns


In [85]:
# Kald main() for at få fat i barra_cov
barra_cov = ECM.main()
#print(barra_cov)

All missing excludes 0.60% of the observations
Filen er indlæst og filtreret succesfuldt.
No size screen
Turnover wo addition/deletion rule: 0.19%
Turnover w  addition/deletion rule: 33.72%


Processerer datoer: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 72/72 [00:00<00:00, 245.41it/s]


In [86]:
wealth_end = pf_set["wealth"]
end = settings["split"]["test_end"]
market_path = "./data_test/market_returns_test.csv"
rente_path = "Data/ff3_m.csv"
risk_free = process_risk_free_rate(rente_path)
market_test = load_and_filter_market_returns_test(market_path)

wealth = wealth_func(wealth_end, end, market_test, risk_free)
wealth = wealth.dropna(subset=['wealth'])
print(wealth)

Filen er indlæst og filtreret succesfuldt.
            eom        wealth    mu_ld1
1001 2009-11-30  1.426752e+09  0.028317
1002 2009-12-31  1.468331e+09 -0.038043
1003 2010-01-31  1.414519e+09  0.034614
1004 2010-02-28  1.465237e+09  0.064595
1005 2010-03-31  1.566419e+09  0.019248
...         ...           ...       ...
1165 2023-07-31  9.975058e+09 -0.019997
1166 2023-08-31  9.779494e+09 -0.048173
1167 2023-09-30  9.330037e+09 -0.029172
1168 2023-10-31  9.065579e+09  0.093442
1169 2023-11-30  1.000000e+10       NaN

[169 rows x 3 columns]


In [87]:
first_cov_date, hp_years, start_oos = define_important_dates(barra_cov, settings)
date_ranges = create_date_ranges(settings, first_cov_date, start_oos, hp_years)
dates_m1 = date_ranges["dates_m1"]
dates_m2 = date_ranges["dates_m2"]
dates_oos = date_ranges["dates_oos"]
dates_hp = date_ranges["dates_hp"]
print(dates_oos)

DatetimeIndex(['2019-12-31', '2020-01-31', '2020-02-29', '2020-03-31',
               '2020-04-30', '2020-05-31', '2020-06-30', '2020-07-31',
               '2020-08-31', '2020-09-30', '2020-10-31', '2020-11-30',
               '2020-12-31', '2021-01-31', '2021-02-28', '2021-03-31',
               '2021-04-30', '2021-05-31', '2021-06-30', '2021-07-31',
               '2021-08-31', '2021-09-30', '2021-10-31', '2021-11-30',
               '2021-12-31', '2022-01-31', '2022-02-28', '2022-03-31',
               '2022-04-30', '2022-05-31', '2022-06-30', '2022-07-31',
               '2022-08-31', '2022-09-30', '2022-10-31', '2022-11-30',
               '2022-12-31', '2023-01-31', '2023-02-28', '2023-03-31',
               '2023-04-30', '2023-05-31', '2023-06-30', '2023-07-31',
               '2023-08-31', '2023-09-30', '2023-10-31'],
              dtype='datetime64[ns]', freq='ME')


In [88]:
def create_lambda_list(chars):
    """
    Opretter en dictionary (`lambda_list`) af aktiers lambda-værdier pr. dato.

    Args:
        chars (pd.DataFrame): Dataframe med aktie-karakteristika inkl. `eom` og `lambda`.

    Returns:
        dict: `lambda_list` hvor nøgler er datoer i formatet 'YYYY-MM-DD' og værdier er dictionaries af {id: lambda}.

    Example: lambda_list = create_lambda_list(chars)
    """
    lambda_dates = chars['eom'].unique()

    lambda_list = {
        pd.to_datetime(d).strftime('%Y-%m-%d'): chars[chars['eom'] == d][['id', 'lambda']].set_index('id')['lambda'].to_dict()
        for d in lambda_dates
    }

    return lambda_list


In [89]:
file_path_usa_dsf = "./Data/usa_dsf.parquet"
file_path_usa = "./Data/usa_rvol.parquet"
file_path_id_test = "./data_test/top_5_percent_ids.csv"
file_path_market_returns = "./Data/market_returns.csv"
output_path_usa_dsf = "./data_test/usa_dsf_test.parquet"
output_path_usa = "./data_test/usa_test.parquet"
output_path_market_returns = "data_test/market_returns_test.csv"
file_path_world_ret = "./Data/world_ret_monthly.csv"
start_date = "2010-01-31"
end_date = "2023-11-30"

output_path = "./data_test/"

file_path_usa_test = "./data_test/usa_test.parquet"
daily_file_path = "./data_test/usa_dsf_test.parquet"
file_path_world_ret = "./data_test/world_ret_test.csv"
risk_free_path = "./data_test/risk_free_test.csv"
market_path = "./data_test/market_returns_test.csv"

rente_path = "Data/ff3_m.csv"


# Kører funktioner fra data_run_files
risk_free = data_run_files.process_risk_free_rate(rente_path, start_date)
h_list = [1]  # Horisonter
data_ret = data_run_files.monthly_returns(risk_free, h_list, output_path_usa)
chars, daily = Prepare_Data.process_all_data(file_path_usa_test, daily_file_path, file_path_world_ret, risk_free_path, market_path)
get_from_path_model = "./data_test/"
chars = prepare_portfolio_data.add_return_predictions(chars, settings, get_from_path_model)
lambda_list = create_lambda_list(chars)

Filtrering udført. Antal rækker efter filtrering: 179
Fil gemt som ./data_test/risk_free_test.csv
All missing excludes 0.63% of the observations
All missing excludes 0.60% of the observations
Filen er indlæst og filtreret succesfuldt.
No size screen
Turnover wo addition/deletion rule: 0.19%
Turnover w  addition/deletion rule: 33.72%


In [11]:
dates_hp = dates_hp[dates_hp >= pd.Timestamp(dates_oos[0].strftime('%Y-%m-%d'))]
dates_oos[0].strftime('%Y-%m-%d')

'2020-01-31'

In [90]:
datecut = pd.to_datetime("2019-12-31")

# Opret dates_oos ved at bruge 'M' (month-end frekvens)
dates_oos = pd.date_range(
    start=pd.to_datetime(f"{start_oos}-01-01") + MonthEnd(0),
    end=(settings["split"]["test_end"] + MonthEnd(0)) - MonthEnd(1),
    freq='M'
)
# Filtrer for datoer efter datecut:
dates_oos = dates_oos[dates_oos > datecut]

# Opret dates_hp på samme måde:
dates_hp = pd.date_range(
    start=pd.to_datetime(f"{min(hp_years)}-01-01") + MonthEnd(0),
    end=(settings["split"]["test_end"] + MonthEnd(0)) - MonthEnd(1),
    freq='M'
)
dates_hp = dates_hp[dates_hp > datecut]

# Hvis dates_m1 allerede er defineret, filtreres de også:
dates_m1 = dates_m1[dates_m1 > datecut]  # Forudsætter, at dates_m1 er en pd.DatetimeIndex eller lignende
dates_m2 = dates_m2[dates_m2 > datecut]

# Filtrer hp_years for at vælge de år, der er større end 2019:
hp_years = [year for year in hp_years if year > 2019]


In [14]:
import copy

data_tc = copy.deepcopy(chars)
cov_list = copy.deepcopy(barra_cov)
lambda_list = copy.deepcopy(lambda_list)
risk_free = copy.deepcopy(risk_free)
features = copy.deepcopy(features)
wealth = copy.deepcopy(wealth)
mu = copy.deepcopy(pf_set['mu'])
gamma_rel = copy.deepcopy(pf_set['gamma_rel'])
dates_full = copy.deepcopy(dates_m2)
lb = copy.deepcopy(pf_set['lb_hor'])
scale = copy.deepcopy(settings['pf_ml']['scale'])
g_vec = copy.deepcopy(settings['pf_ml']['g_vec'])
p_vec = copy.deepcopy(settings['pf_ml']['p_vec'])
l_vec = copy.deepcopy(settings['pf_ml']['l_vec'])
orig_feat = copy.deepcopy(settings['pf_ml']['orig_feat'])
iter = copy.deepcopy(10)
rff_feat = copy.deepcopy(True)
seed = copy.deepcopy(settings['seed_no'])


# Funktioner

In [313]:
def create_lambda_list(chars):
    """
    Opretter en dictionary (`lambda_list`) af aktiers lambda-værdier pr. dato.

    Args:
        chars (pd.DataFrame): Dataframe med aktie-karakteristika inkl. `eom` og `lambda`.

    Returns:
        dict: `lambda_list` hvor nøgler er datoer i formatet 'YYYY-MM-DD' og værdier er dictionaries af {id: lambda}.

    Example: lambda_list = create_lambda_list(chars)
    """
    lambda_dates = chars['eom'].unique()

    lambda_list = {
        pd.to_datetime(d).strftime('%Y-%m-%d'): chars[chars['eom'] == d][['id', 'lambda']].set_index('id')['lambda'].to_dict()
        for d in lambda_dates
    }

    return lambda_list


In [50]:

# Funktion til beregning af M
def m_func(w, mu, rf, sigma_gam, gam, lambda_mat, iter): ## ændret denne i port_8 push, hvis problemer
    # Konverter inputs til NumPy-arrays, hvis de ikke allerede er det
    sigma_gam = np.array(sigma_gam)
    lambda_mat = np.array(lambda_mat)

    n = lambda_mat.shape[0]
    print("m_func: n =", n)

    g_bar = np.ones(n)
    mu_bar_vec = np.ones(n) * (1 + rf + mu)

    # Beregn sigma_gr
    sigma_gr = (1 + rf + mu) ** -2 * (np.outer(mu_bar_vec, mu_bar_vec) + sigma_gam / gam)
#    print("m_func: sigma_gr shape:", sigma_gr.shape)

    # Beregn lambda^(-0.5), da lambda er diagonal
    lamb_neg05 = np.diag(np.diag(lambda_mat) ** -0.5)
#    print("m_func: lamb_neg05 shape:", lamb_neg05.shape)

    # Beregn x
    x = (w ** -1) * (lamb_neg05 @ sigma_gam @ lamb_neg05)
#    print("m_func: x shape:", x.shape)

    # Beregn y
    y = np.diag(1 + np.diag(sigma_gr))
#    print("m_func: y shape:", y.shape)

    sigma_hat = x + np.diag(1 + g_bar)
#    print("m_func: sigma_hat shape:", sigma_hat.shape)

    # Brug C++-implementeringen af sqrtm, og tag realdelen
    sqrt_term = np.real(sqrtm_cpp.sqrtm_cpp(sigma_hat @ sigma_hat - 4 * np.eye(n)))
#    print("m_func: sqrt_term shape:", sqrt_term.shape)

    m_tilde = 0.5 * (sigma_hat - sqrt_term)
#    print("m_func: initial m_tilde shape:", m_tilde.shape)

    # Iteration på F med elementvis multiplikation (hvis det er tilsigtet)
    for i in range(iter):
        inv_arg = x + y - m_tilde * sigma_gr  # Elementvis multiplikation
        #print(f"m_func: Iteration {i}, inv_arg shape:", inv_arg.shape)
        m_tilde = np.linalg.inv(inv_arg)
        #print(f"m_func: Iteration {i}, m_tilde shape:", m_tilde.shape)

    # Beregn elementvis kvadratrod af lambda (dette giver en diagonal matrix)
    lambda_sqrt = np.diag(np.sqrt(np.diag(lambda_mat)))

    result = lamb_neg05 @ m_tilde @ lambda_sqrt
    #print("m_func: result shape:", result.shape)
    return result

In [22]:
def rff(X, p=None, g=None, W=None):
    """
    Genererer Random Fourier Features (RFF) fra input-data.

    Args:
        X (np.ndarray): Input-data som en matrix (n x d).
        p (int): Antal Fourier-funktioner (skal være delbart med 2).
        g (float): Varians/skala for vægtmatrixen W.
        W (np.ndarray, optional): Foruddefineret vægtmatrix (d x p/2). Hvis None, genereres tilfældige vægte.

    Returns:
        dict: En dictionary med:
            - "W": Vægtmatrixen (d x p/2).
            - "X_cos": Cosinus-transformeret data (n x p/2).
            - "X_sin": Sinus-transformeret data (n x p/2).

    Example:
        X = np.random.rand(100, 5)  # 100 eksempler, 5 features
        rff_result = rff(X, p=10, g=1.0)
        print("W:", rff_result["W"].shape)
        print("X_cos:", rff_result["X_cos"].shape)
        print("X_sin:", rff_result["X_sin"].shape)
    """

    # Tjek at p er delbart med 2
    if p % 2 != 0:
        raise ValueError("p skal være delbart med 2!")

    # Generér vægtmatrix W, hvis ikke angivet
    if W is None:
        k = X.shape[1]  # Antal features (dimensionen af X)
        W = np.random.multivariate_normal(
            mean=np.zeros(k),
            cov=g * np.eye(k),
            size=p // 2
        ).T  # Transponér så dimensionerne matcher (k x p/2)

    # Beregn X_new = X * W
    X_new = np.dot(X, W)  # Matrix-multiplikation

    # Transformer data til cosinus og sinus
    X_cos = np.cos(X_new)
    X_sin = np.sin(X_new)

    # Returnér vægte og transformeret data
    return {"W": W, "X_cos": X_cos, "X_sin": X_sin}


In [82]:
chars["eom"] 

0       2010-12-31
1       2011-01-31
2       2011-02-28
3       2011-03-31
4       2011-04-30
           ...    
26644   2023-06-30
26645   2023-07-31
26646   2023-08-31
26647   2023-09-30
26648   2023-10-31
Name: eom, Length: 26649, dtype: datetime64[ns]

In [91]:
def pfml_input_fun(data_tc, cov_list, lambda_list, gamma_rel, wealth, mu, dates, lb, scale, 
                   risk_free, features, rff_feat, seed, p_max, g, add_orig, iter, balanced):

    # --- Lookback-datoer ---
    min_date = min(dates)
    max_date = max(dates)
    start_date = (min_date + pd.Timedelta(days=1)) - MonthEnd(lb+1)
    dates_lb = pd.date_range(start=start_date, end=max_date, freq=MonthEnd())
    
    # --- Oprettelse af Random Fourier Features ---
    if rff_feat:
        np.random.seed(seed)
        X_features = data_tc[features].values
        rff_x = rff(X_features, p=p_max, g=g)
        rff_w = rff_x['W']
        X_cos = rff_x['X_cos']
        X_sin = rff_x['X_sin']
        rff_features = np.hstack([X_cos, X_sin])
        num = p_max // 2
        rff_colnames = [f"rff{i}_cos" for i in range(1, num+1)] + [f"rff{i}_sin" for i in range(1, num+1)]
        rff_df = pd.DataFrame(rff_features, columns=rff_colnames, index=data_tc.index)
        data = pd.concat([data_tc[['id', 'eom', 'valid', 'ret_ld1', 'tr_ld0', 'mu_ld0']].reset_index(drop=True),
                          rff_df.reset_index(drop=True)], axis=1)
        feat_new = list(rff_df.columns)
        if add_orig:
            data = pd.concat([data, data_tc[features].reset_index(drop=True)], axis=1)
            feat_new = feat_new + features
    else:
        cols = ['id', 'eom', 'valid', 'ret_ld1', 'tr_ld0', 'mu_ld0'] + features
        data = data_tc[cols].copy()
        feat_new = features.copy()
    
    feat_cons = feat_new + ['constant']
    
    # --- Tilføjelse af scales ---
    if scale:
        scales_list = []
        for d in dates_lb:
            d_str = d.strftime('%Y-%m-%d')
            sigma = General_Functions.create_cov(cov_list[d_str])
            if hasattr(sigma, 'values'):
                sigma_vals = sigma.values
                ids_sigma = sigma.index.astype(float)
            else:
                sigma_vals = sigma
                ids_sigma = np.arange(sigma.shape[0])
            diag_vol = np.sqrt(np.diag(sigma_vals))
            df_scales = pd.DataFrame({
                'id': ids_sigma,
                'eom': d,
                'vol_scale': diag_vol
            })
            scales_list.append(df_scales)
        scales_df = pd.concat(scales_list, ignore_index=True)
        data = pd.merge(data, scales_df, on=['id', 'eom'], how='left')
        data['vol_scale'] = data.groupby('eom')['vol_scale'].transform(lambda x: x.fillna(x.median()))
    
    # --- Justering ved balanced panel ---
    if balanced:
        for col in feat_new:
            data[col] = data.groupby('eom')[col].transform(lambda x: x - x.mean())
        data['constant'] = 1
        for col in feat_cons:
            data[col] = data.groupby('eom')[col].transform(lambda x: x * np.sqrt(1 / np.sum(x**2)) if np.sum(x**2) != 0 else 0)
    
    # --- Beregning af signaler og realiseringer for hvert dato ---
    reals_dict = {}
    signal_t_dict = {}
    
    for d in dates:
        if d.year % 10 == 0 and d.month == 1:
            print(f"--> PF-ML inputs: {d}")
            
        data_ret = data[(data['valid'] == True) & (data['eom'] == d)][['id', 'ret_ld1']]
        ids = data_ret['id'].unique()  # ids som ints
        n = len(ids)
        r_vec = data_ret['ret_ld1'].values
        print("d", d)
        d_str = d.strftime('%Y-%m-%d')
        print("d_str", d_str)
        
        # Brug de korrekte key-typer (ints) til både sigma og lambda
        sigma = General_Functions.create_cov(cov_list[d_str], ids=ids)
        lambda_series = pd.Series(lambda_list[d_str])
        lambda_mat = General_Functions.create_lambda(lambda_series, ids=ids)
        
        w = wealth.loc[wealth['eom'] == d, 'wealth'].iloc[0]
        rf = risk_free.loc[risk_free['eom'] == d, 'rf'].iloc[0]
        m = m_func(w=w, mu=mu, rf=rf, sigma_gam=sigma * gamma_rel, gam=gamma_rel,
                   lambda_mat=lambda_mat, iter=iter)
        
        # Brug MonthEnd til at beregne lower_bound
        lower_bound = d - MonthEnd(lb)
        data_sub = data[(data['id'].isin(ids)) & (data['eom'] >= lower_bound) & (data['eom'] <= d)].copy()
        
        if not balanced:
            for col in feat_new:
                data_sub[col] = data_sub.groupby('eom')[col].transform(lambda x: x - x.mean())
            data_sub['constant'] = 1
            for col in feat_cons:
                data_sub[col] = data_sub.groupby('eom')[col].transform(lambda x: x * np.sqrt(1 / np.sum(x**2)) if np.sum(x**2) != 0 else 0)
        
        data_sub = data_sub.sort_values(by=['eom', 'id'], ascending=[False, True])
        groups = {
            (k.strftime('%Y-%m-%d') if isinstance(k, pd.Timestamp) else str(k)): group
            for k, group in data_sub.groupby('eom')
        }
        
        # Beregn signaler med diagnose: udskriv kun hvis NaN, Inf, tom DataFrame eller tomt index
        signals = {}
        for eom_val, group in groups.items():
            if group.empty or group.index.empty:
                print(f"Advarsel: Gruppe for {eom_val} er tom!")
            s = group[feat_cons].values
            if scale:
                s = np.diag(1 / group['vol_scale'].values) @ s
            # Tjek for NaN eller Inf
            if np.isnan(s).any():
                print(f"Advarsel: Signal for {eom_val} indeholder NaN. Form: {s.shape}")
            if np.isinf(s).any():
                print(f"Advarsel: Signal for {eom_val} indeholder Inf. Form: {s.shape}")
            signals[eom_val] = s
        
        d_key = d.strftime('%Y-%m-%d') if isinstance(d, pd.Timestamp) else str(d)
        signal_current = signals.get(d_key, None)
        if signal_current is None:
            print(f"Advarsel: Ingen signal fundet for {d_key}.")
            continue
        elif np.isnan(signal_current).any() or np.isinf(signal_current).any():
            print(f"Advarsel: Signal for {d_key} indeholder NaN eller Inf, springer denne dato over.")
            continue
        
        # Beregn gtm for hver gruppe
        gtm = {}
        for eom_val, group in groups.items():
            gt = (1 + group['tr_ld0']) / (1 + group['mu_ld0'])
            gt = gt.fillna(1).values
            gtm[eom_val] = m @ np.diag(gt)
        
        # Aggregér gtm over lookback-perioden
        n_stocks = n
        gtm_agg = {}
        gtm_agg_l1 = {}
        if d_key in signals:
            group_d = groups.get(d_key, None)
        gtm_agg[d_key] = np.eye(n_stocks)
        gtm_agg_l1[d_key] = np.eye(n_stocks)
        for i in range(1, lb + 1):
            d_i = (d - MonthEnd(i)).strftime('%Y-%m-%d')
            if d_i in gtm:
                gtm_agg[d_i] = gtm_agg[list(gtm_agg.keys())[-1]] @ gtm[d_i]
            else:
                gtm_agg[d_i] = gtm_agg[list(gtm_agg.keys())[-1]]
            d_i_l1 = (d - MonthEnd(i+1)).strftime('%Y-%m-%d')
            if d_i_l1 in gtm:
                gtm_agg_l1[d_i] = gtm_agg_l1[list(gtm_agg_l1.keys())[-1]] @ gtm[d_i_l1]
            else:
                gtm_agg_l1[d_i] = gtm_agg_l1[list(gtm_agg_l1.keys())[-1]]
        
        # Summering over lookback: opbyg omega og konstanter
        omega_sum = None
        const_sum = None
        omega_l1_sum = None
        const_l1_sum = None
        for i in range(0, lb + 1):
            d_i = (d - MonthEnd(i)).strftime('%Y-%m-%d')
            s_i = signals.get(d_i, None)
            # Hvis s_i er None, brug en nulmatrix med form (n_stocks, antal feat_cons)
            if s_i is None:
                print(f"Advarsel: Ingen signal fundet for lookback-dato {d_i}")
                term = np.zeros((n_stocks, len(feat_cons)))
            else:
                term = gtm_agg.get(d_i, np.eye(n_stocks)) @ s_i
            if omega_sum is None:
                omega_sum = term
                const_sum = gtm_agg.get(d_i, np.eye(n_stocks))
            else:
                omega_sum = omega_sum + term
                const_sum = const_sum + gtm_agg.get(d_i, np.eye(n_stocks))
            
            d_i_l1 = (d - MonthEnd(i+1)).strftime('%Y-%m-%d')
            s_i_l1 = signals.get(d_i_l1, None)
            if s_i_l1 is None:
                print(f"Advarsel: Ingen signal fundet for lookback-dato {d_i_l1}")
                term_l1 = np.zeros((n_stocks, len(feat_cons)))
            else:
                term_l1 = gtm_agg_l1.get(d_i_l1, np.eye(n_stocks)) @ s_i_l1
            if omega_l1_sum is None:
                omega_l1_sum = term_l1
                const_l1_sum = gtm_agg_l1.get(d_i_l1, np.eye(n_stocks))
            else:
                omega_l1_sum = omega_l1_sum + term_l1
                const_l1_sum = const_l1_sum + gtm_agg_l1.get(d_i_l1, np.eye(n_stocks))
        
        # Løs de lineære systemer
        omega_final = np.linalg.solve(const_sum, omega_sum)
        omega_l1_final = np.linalg.solve(const_l1_sum, omega_l1_sum)
        
        # Beregn gt for den aktuelle gruppe (d)
        if d_key in signals:
            group_d = groups.get(d_key, None)
            if group_d is None:
                gt_vec = np.ones(n_stocks)
            else:
                gt_vec = ((1 + group_d['tr_ld0']) / (1 + group_d['mu_ld0'])).values
            gt_mat = np.diag(gt_vec)
        else:
            gt_mat = np.eye(n_stocks)
        
        omega_chg = omega_final - gt_mat @ omega_l1_final
        
        # --- Realiseringer ---
        r_tilde = omega_final.T @ r_vec
        risk_val = gamma_rel * (omega_final.T @ sigma @ omega_final)
        tc_val = w * (omega_chg.T @ lambda_mat @ omega_chg)
        denom = risk_val + tc_val
        
        reals = {
            "r_tilde": r_tilde.item() if np.isscalar(r_tilde) or r_tilde.size == 1 else r_tilde,
            "denom": denom.item() if np.isscalar(denom) or denom.size == 1 else denom,
            "risk": risk_val.item() if np.isscalar(risk_val) or risk_val.size == 1 else risk_val,
            "tc": tc_val.item() if np.isscalar(tc_val) or tc_val.size == 1 else tc_val
        }
        
        reals_dict[d_key] = reals
        signal_t_dict[d_key] = signals.get(d_key, None)
    
    return {"reals": reals_dict, "signal_t": signal_t_dict, "rff_w": rff_w if rff_feat else None}

In [ ]:
## udprint datoer

In [94]:
from pandas.tseries.offsets import MonthEnd

def pfml_input_fun(data_tc, cov_list, lambda_list, gamma_rel, wealth, mu, dates, lb, scale, 
                   risk_free, features, rff_feat, seed, p_max, g, add_orig, iter, balanced):

    # --- Lookback-datoer ---
    min_date = min(dates)
    max_date = max(dates)
    start_date = (min_date + pd.Timedelta(days=1)) - MonthEnd(lb+1)
    dates_lb = pd.date_range(start=start_date, end=max_date, freq=MonthEnd())
    
    # --- Oprettelse af Random Fourier Features ---
    if rff_feat:
        np.random.seed(seed)
        X_features = data_tc[features].values
        rff_x = rff(X_features, p=p_max, g=g)
        rff_w = rff_x['W']
        X_cos = rff_x['X_cos']
        X_sin = rff_x['X_sin']
        rff_features = np.hstack([X_cos, X_sin])
        num = p_max // 2
        rff_colnames = [f"rff{i}_cos" for i in range(1, num+1)] + [f"rff{i}_sin" for i in range(1, num+1)]
        rff_df = pd.DataFrame(rff_features, columns=rff_colnames, index=data_tc.index)
        data = pd.concat([data_tc[['id', 'eom', 'valid', 'ret_ld1', 'tr_ld0', 'mu_ld0']].reset_index(drop=True),
                          rff_df.reset_index(drop=True)], axis=1)
        feat_new = list(rff_df.columns)
        if add_orig:
            data = pd.concat([data, data_tc[features].reset_index(drop=True)], axis=1)
            feat_new = feat_new + features
    else:
        cols = ['id', 'eom', 'valid', 'ret_ld1', 'tr_ld0', 'mu_ld0'] + features
        data = data_tc[cols].copy()
        feat_new = features.copy()
    
    feat_cons = feat_new + ['constant']
    
    # --- Tilføjelse af scales ---
    if scale:
        scales_list = []
        for d in dates_lb:
            d_str = d.strftime('%Y-%m-%d')
            sigma = General_Functions.create_cov(cov_list[d_str])
            if hasattr(sigma, 'values'):
                sigma_vals = sigma.values
                ids_sigma = sigma.index.astype(float)
            else:
                sigma_vals = sigma
                ids_sigma = np.arange(sigma.shape[0])
            diag_vol = np.sqrt(np.diag(sigma_vals))
            df_scales = pd.DataFrame({
                'id': ids_sigma,
                'eom': d,
                'vol_scale': diag_vol
            })
            scales_list.append(df_scales)
        scales_df = pd.concat(scales_list, ignore_index=True)
        data = pd.merge(data, scales_df, on=['id', 'eom'], how='left')
        data['vol_scale'] = data.groupby('eom')['vol_scale'].transform(lambda x: x.fillna(x.median()))
    
    # --- Justering ved balanced panel ---
    if balanced:
        for col in feat_new:
            data[col] = data.groupby('eom')[col].transform(lambda x: x - x.mean())
        data['constant'] = 1
        for col in feat_cons:
            data[col] = data.groupby('eom')[col].transform(lambda x: x * np.sqrt(1 / np.sum(x**2)) if np.sum(x**2) != 0 else 0)
    
    # --- Beregning af signaler og realiseringer for hvert dato ---
    reals_dict = {}
    signal_t_dict = {}
    
    for d in dates:
        if d.year % 10 == 0 and d.month == 1:
            print(f"--> PF-ML inputs: {d}")
            
        data_ret = data[(data['valid'] == True) & (data['eom'] == d)][['id', 'ret_ld1']]
        ids = data_ret['id'].unique()  # ids som ints
        n = len(ids)
        r_vec = data_ret['ret_ld1'].values
        print("d", d)
        d_str = d.strftime('%Y-%m-%d')
        print("d_str", d_str)
        
        # Brug de korrekte key-typer til både sigma og lambda
        sigma = General_Functions.create_cov(cov_list[d_str], ids=ids)
        lambda_series = pd.Series(lambda_list[d_str])
        lambda_mat = General_Functions.create_lambda(lambda_series, ids=ids)
        
        w = wealth.loc[wealth['eom'] == d, 'wealth'].iloc[0]
        rf = risk_free.loc[risk_free['eom'] == d, 'rf'].iloc[0]
        m = m_func(w=w, mu=mu, rf=rf, sigma_gam=sigma * gamma_rel, gam=gamma_rel,
                   lambda_mat=lambda_mat, iter=iter)
        
        # Brug MonthEnd til at beregne lower_bound
        lower_bound = d - MonthEnd(lb)
        data_sub = data[(data['id'].isin(ids)) & (data['eom'] >= lower_bound) & (data['eom'] <= d)].copy()
        
        if not balanced:
            for col in feat_new:
                data_sub[col] = data_sub.groupby('eom')[col].transform(lambda x: x - x.mean())
            data_sub['constant'] = 1
            for col in feat_cons:
                data_sub[col] = data_sub.groupby('eom')[col].transform(lambda x: x * np.sqrt(1 / np.sum(x**2)) if np.sum(x**2) != 0 else 0)
        
        data_sub = data_sub.sort_values(by=['eom', 'id'], ascending=[False, True])
        groups = {
            (k.strftime('%Y-%m-%d') if isinstance(k, pd.Timestamp) else str(k)): group
            for k, group in data_sub.groupby('eom')
        }
        
        # Beregn signaler med diagnose: udskriv kun hvis NaN, Inf, tom DataFrame eller tomt index
        signals = {}
        for eom_val, group in groups.items():
            if group.empty or group.index.empty:
                print(f"Advarsel: Gruppe for {eom_val} er tom!")
            s = group[feat_cons].values
            if scale:
                s = np.diag(1 / group['vol_scale'].values) @ s
            # Tjek for NaN eller Inf
            if np.isnan(s).any():
                print(f"Advarsel: Signal for {eom_val} indeholder NaN. Form: {s.shape}")
            if np.isinf(s).any():
                print(f"Advarsel: Signal for {eom_val} indeholder Inf. Form: {s.shape}")
            signals[eom_val] = s
        
        d_key = d.strftime('%Y-%m-%d') if isinstance(d, pd.Timestamp) else str(d)
        signal_current = signals.get(d_key, None)
        if signal_current is None:
            print(f"Advarsel: Ingen signal fundet for {d_key}.")
            continue
        elif np.isnan(signal_current).any() or np.isinf(signal_current).any():
            print(f"Advarsel: Signal for {d_key} indeholder NaN eller Inf, springer denne dato over.")
            continue
        
        # Beregn gtm for hver gruppe
        gtm = {}
        for eom_val, group in groups.items():
            gt = (1 + group['tr_ld0']) / (1 + group['mu_ld0'])
            gt = gt.fillna(1).values
            gtm[eom_val] = m @ np.diag(gt)
        
        # Aggregér gtm over lookback-perioden
        n_stocks = n
        gtm_agg = {}
        gtm_agg_l1 = {}
        if d_key in signals:
            group_d = groups.get(d_key, None)
        gtm_agg[d_key] = np.eye(n_stocks)
        gtm_agg_l1[d_key] = np.eye(n_stocks)
        for i in range(1, lb + 1):
            d_i = (d - MonthEnd(i)).strftime('%Y-%m-%d')
            # DEBUG: Udskriv kun unikke eom-datoer for den lookback-dato
            lookback_date = pd.to_datetime(d_i)
            
            if d_i in gtm:
                gtm_agg[d_i] = gtm_agg[list(gtm_agg.keys())[-1]] @ gtm[d_i]
            else:
                gtm_agg[d_i] = gtm_agg[list(gtm_agg.keys())[-1]]
            d_i_l1 = (d - MonthEnd(i+1)).strftime('%Y-%m-%d')
            if d_i_l1 in gtm:
                gtm_agg_l1[d_i] = gtm_agg_l1[list(gtm_agg_l1.keys())[-1]] @ gtm[d_i_l1]
            else:
                gtm_agg_l1[d_i] = gtm_agg_l1[list(gtm_agg_l1.keys())[-1]]
        
        # Summering over lookback: opbyg omega og konstanter
        omega_sum = None
        const_sum = None
        omega_l1_sum = None
        const_l1_sum = None
        for i in range(0, lb + 1):
            d_i = (d - MonthEnd(i)).strftime('%Y-%m-%d')
            s_i = signals.get(d_i, None)
            if s_i is None:
                print(f"Advarsel: Ingen signal fundet for lookback-dato {d_i}")
                term = np.zeros((n_stocks, len(feat_cons)))
            else:
                term = gtm_agg.get(d_i, np.eye(n_stocks)) @ s_i
            if omega_sum is None:
                omega_sum = term
                const_sum = gtm_agg.get(d_i, np.eye(n_stocks))
            else:
                omega_sum = omega_sum + term
                const_sum = const_sum + gtm_agg.get(d_i, np.eye(n_stocks))
            
            d_i_l1 = (d - MonthEnd(i+1)).strftime('%Y-%m-%d')
            s_i_l1 = signals.get(d_i_l1, None)
            if s_i_l1 is None:
                print(f"Advarsel: Ingen signal fundet for lookback-dato {d_i_l1}")
                term_l1 = np.zeros((n_stocks, len(feat_cons)))
            else:
                term_l1 = gtm_agg_l1.get(d_i_l1, np.eye(n_stocks)) @ s_i_l1
            if omega_l1_sum is None:
                omega_l1_sum = term_l1
                const_l1_sum = gtm_agg_l1.get(d_i_l1, np.eye(n_stocks))
            else:
                omega_l1_sum = omega_l1_sum + term_l1
                const_l1_sum = const_l1_sum + gtm_agg_l1.get(d_i_l1, np.eye(n_stocks))
        
        # Løs de lineære systemer
        omega_final = np.linalg.solve(const_sum, omega_sum)
        omega_l1_final = np.linalg.solve(const_l1_sum, omega_l1_sum)
        
        # Beregn gt for den aktuelle gruppe (d)
        if d_key in signals:
            group_d = groups.get(d_key, None)
            if group_d is None:
                gt_vec = np.ones(n_stocks)
            else:
                gt_vec = ((1 + group_d['tr_ld0']) / (1 + group_d['mu_ld0'])).values
            gt_mat = np.diag(gt_vec)
        else:
            gt_mat = np.eye(n_stocks)
        
        omega_chg = omega_final - gt_mat @ omega_l1_final
        
        # --- Realiseringer ---
        r_tilde = omega_final.T @ r_vec
        risk_val = gamma_rel * (omega_final.T @ sigma @ omega_final)
        tc_val = w * (omega_chg.T @ lambda_mat @ omega_chg)
        denom = risk_val + tc_val
        
        reals = {
            "r_tilde": r_tilde.item() if np.isscalar(r_tilde) or r_tilde.size == 1 else r_tilde,
            "denom": denom.item() if np.isscalar(denom) or denom.size == 1 else denom,
            "risk": risk_val.item() if np.isscalar(risk_val) or risk_val.size == 1 else risk_val,
            "tc": tc_val.item() if np.isscalar(tc_val) or tc_val.size == 1 else tc_val
        }
        
        reals_dict[d_key] = reals
        signal_t_dict[d_key] = signals.get(d_key, None)
    
    return {"reals": reals_dict, "signal_t": signal_t_dict, "rff_w": rff_w if rff_feat else None}


In [95]:
pfml_input_fun(
    data_tc=chars,
    cov_list=barra_cov,
    lambda_list=lambda_list,
    gamma_rel=pf_set['gamma_rel'],
    wealth=wealth,
    mu=pf_set['mu'],
    dates=dates_m2,            # i stedet for dates_full
    lb=pf_set['lb_hor'],
    scale=settings['pf_ml']['scale'],
    risk_free=risk_free,
    features=features,
    rff_feat=True,
    seed=settings['seed_no'],
    p_max=256,                 # fast p_max
    g=0.049787068367863944,     # fast g
    add_orig=settings['pf_ml']['orig_feat'],  # i stedet for orig_feat
    iter=10,
    balanced=True              # eller False, alt efter hvad du ønsker
)


--> PF-ML inputs: 2020-01-31 00:00:00
d 2020-01-31 00:00:00
d_str 2020-01-31
m_func: n = 172
Advarsel: Ingen signal fundet for lookback-dato 2019-01-31
d 2020-02-29 00:00:00
d_str 2020-02-29
m_func: n = 172
Advarsel: Ingen signal fundet for lookback-dato 2019-02-28
d 2020-03-31 00:00:00
d_str 2020-03-31
m_func: n = 172
Advarsel: Ingen signal fundet for lookback-dato 2019-03-31
d 2020-04-30 00:00:00
d_str 2020-04-30
m_func: n = 172
Advarsel: Ingen signal fundet for lookback-dato 2019-04-30
d 2020-05-31 00:00:00
d_str 2020-05-31
m_func: n = 172
Advarsel: Ingen signal fundet for lookback-dato 2019-05-31
d 2020-06-30 00:00:00
d_str 2020-06-30
m_func: n = 172
Advarsel: Ingen signal fundet for lookback-dato 2019-06-30
d 2020-07-31 00:00:00
d_str 2020-07-31
m_func: n = 172
Advarsel: Ingen signal fundet for lookback-dato 2019-07-31
d 2020-08-31 00:00:00
d_str 2020-08-31
m_func: n = 172
Advarsel: Ingen signal fundet for lookback-dato 2019-08-31
d 2020-09-30 00:00:00
d_str 2020-09-30
m_func: n =

{'reals': {'2020-01-31': {'r_tilde': array([ 7.00877113e-01, -1.14687713e+00, -2.52092373e+00, -1.74065296e+00,
           1.65931859e+00,  1.42414476e+00, -1.45628571e+00, -1.22894703e+00,
          -5.88332892e-01, -3.62133516e-01,  7.67191096e-01, -4.34917054e-01,
           2.08538480e+00, -1.53507741e+00, -2.14110064e+00, -2.17594408e+00,
          -1.74665943e+00, -4.02174039e+00,  2.11918861e+00, -4.64475633e-01,
          -9.98343596e-01, -1.12423390e+00, -3.17583829e+00, -1.45320299e+00,
           2.00320968e-01, -2.32262363e+00,  3.94365105e+00, -2.40834444e+00,
          -1.21707044e+00, -8.08684453e-02,  1.67530722e-01, -2.73809567e+00,
          -1.15991887e+00,  1.77511896e+00,  2.93792755e-01, -7.90470418e-01,
          -1.59354253e+00, -1.85646150e-01, -2.53776927e+00, -2.38509889e+00,
           7.01425911e-01,  5.80766596e-01, -4.79013896e+00, -3.55181560e+00,
          -3.36696881e-01, -6.97824942e-01,  8.84006234e-01,  4.64445886e-01,
          -1.59569550e+00,  1.

In [97]:
chars["eom"].unique

<bound method Series.unique of 0       2010-12-31
1       2011-01-31
2       2011-02-28
3       2011-03-31
4       2011-04-30
           ...    
26644   2023-06-30
26645   2023-07-31
26646   2023-08-31
26647   2023-09-30
26648   2023-10-31
Name: eom, Length: 26649, dtype: datetime64[ns]>

In [63]:
def pfml_feat_fun(p, orig_feat, features=None):
    """
    Returnerer en liste af feature-navne.
    
    Parametre:
      p         : Antal RFF-features (skal være deleligt med 2). Hvis p er 0, bruges ingen RFF-features.
      orig_feat : Bool – om de originale features skal inkluderes.
      features  : Liste af originale feature-navne. Skal gives, hvis orig_feat er True.
    
    Returnerer:
      En liste med feature-navne.
    """
    feat = ["constant"]
    if p != 0:
        half_p = p // 2
        feat.extend([f"rff{i}_cos" for i in range(1, half_p + 1)])
        feat.extend([f"rff{i}_sin" for i in range(1, half_p + 1)])
    if orig_feat and features is not None:
        feat.extend(features)
    return feat


def denom_sum_fun(train):
    """
    Beregner summen af 'denom'-værdier fra en liste af træningsdata.
    
    Parametre:
      train : Liste af ordbøger, hvor hver ordbog indeholder en nøgle 'denom'.
      
    Returnerer:
      Summen af alle 'denom'-værdier.
    """
    # Hvis 'denom' er et tal eller en numpy-array, summeres de med sum().
    return sum(x['denom'] for x in train)


In [64]:
import numpy as np
import pandas as pd

def pfml_search_coef(pfml_input, p_vec, l_vec, hp_years, orig_feat):
    """
    Udfører en hyperparameter-søgning efter de bedste koefficienter.

    Parametre:
      pfml_input : Ordbog med mindst nøglen "reals". 
                   pfml_input["reals"] skal være en ordbog, hvor nøglerne er datoer (som strings)
                   og værdier er ordbøger med 'r_tilde' (pandas Series) og 'denom' (pandas DataFrame).
      p_vec      : Liste af heltal, som angiver antallet af RFF-features (skal være deleligt med 2).
      l_vec      : Liste af regulariseringsparametre.
      hp_years   : Liste af hp-år (for eksempel [2016, 2017, ...]). Forudsættes at de er numeriske årstal.
      orig_feat  : Bool – om de originale features skal medtages (bruges i pfml_feat_fun).

    Returnerer:
      coef_list : En ordbog med koefficienter for hvert hp_year. For hvert hp_year indeholder den en 
                  ordbog med nøgler svarende til p-værdier, og for hver p en ordbog med nøgler for 
                  hver l-værdi.
    """
    # Konverter nøglerne (datoer) til datetime-objekter
    reals = pfml_input["reals"]
    # Vi antager, at nøglerne kan konverteres med pd.to_datetime
    d_all = {pd.to_datetime(k): k for k in reals.keys()}
    
    # Bestem end_bef = min(hp_years) - 1 (fx hvis det mindste hp_year er 2016, så er end_bef 2015)
    end_bef = min(hp_years) - 1
    # Vælg træningsdata før: datoer < (min(hp_years)-2)-12-31
    cutoff_date = pd.to_datetime(f"{end_bef - 1}-12-31")
    train_bef = {k: reals[d_all[k]] for k in d_all if k < cutoff_date}
    
    # Beregn sum af r_tilde over træningssættet.
    # Her antages det, at hver x['r_tilde'] er en pandas Series med index svarende til feature-navne.
    r_tilde_list = [item['r_tilde'] for item in train_bef.values()]
    # Summering: her benyttes sum() til pandas Series (forudsætter, at listen ikke er tom)
    r_tilde_sum = sum(r_tilde_list)
    
    # Tilsvarende: sum af 'denom'-matricer (antages at være pandas DataFrames med feature-navne som index/columns)
    denom_list = [item['denom'] for item in train_bef.values()]
    denom_raw_sum = sum(denom_list)
    
    # n er antallet af observationer i train_bef
    n = len(train_bef)
    
    # Sorter hp_years, og opret en ordbog til at holde koefficienterne
    hp_years = sorted(hp_years)
    coef_list = {}
    
    # For hvert hp_year
    for hp in hp_years:
        # Udvælg nye træningsobservationer: datoer i intervallet
        # [ (hp-2)-12-31, (hp-1)-11-30 ]
        lower_bound = pd.to_datetime(f"{hp - 2}-12-31")
        upper_bound = pd.to_datetime(f"{hp - 1}-11-30")
        
        train_new = {k: reals[d_all[k]] for k in d_all if lower_bound <= k <= upper_bound}
        
        # Opdater antallet af observationer
        n += len(train_new)
        # Opdater r_tilde_sum med de nye r_tilde-værdier
        r_tilde_new = sum([item['r_tilde'] for item in train_new.values()]) if len(train_new) > 0 else 0
        r_tilde_sum = r_tilde_sum + r_tilde_new
        
        # Opdater denom-summen
        denom_raw_new = sum([item['denom'] for item in train_new.values()]) if len(train_new) > 0 else 0
        denom_raw_sum = denom_raw_sum + denom_raw_new
        
        # For hvert p i p_vec, beregn de tilhørende koefficienter
        coef_by_hp = {}
        for p in p_vec:
            # Hent feature-navnene givet p og om de originale features skal medtages
            feat_p = pfml_feat_fun(p, orig_feat)
            # r_tilde_sub: udtræk de features fra r_tilde_sum og divider med n
            r_tilde_sub = r_tilde_sum.loc[feat_p] / n
            # denom_sub: udtræk de relevante rækker og kolonner fra denom_raw_sum og divider med n
            denom_sub = denom_raw_sum.loc[feat_p, feat_p] / n
            
            results = {}
            for l in l_vec:
                # Tilføj regularisering: l*I. Her antages, at antallet af features er len(feat_p) = p+1 (i henhold til R-koden)
                M = denom_sub + l * np.eye(len(feat_p))
                # Løs det lineære system: M * coef = r_tilde_sub
                coef = np.linalg.solve(M.values, r_tilde_sub.values)
                # Gem resultatet som en pandas Series med index = feat_p
                results[l] = pd.Series(coef, index=feat_p)
            # Gem resultaterne for den aktuelle p-værdi
            coef_by_hp[p] = results
        # Gem koefficienterne for det aktuelle hp_year, brug hp som streng
        coef_list[str(hp)] = coef_by_hp
        
    return coef_list


In [65]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import MonthEnd

def pfml_hp_reals_fun(pfml_input, hp_coef, p_vec, l_vec, hp_years, orig_feat):
    """
    Beregner realiseret nytte (realized utility) for hvert p i hvert hp_year.

    Parametre:
      pfml_input : Ordbog med nøglen "reals". pfml_input["reals"] skal være en ordbog, hvor
                   nøglerne er dato-strenge, og værdier er ordbøger med 'r_tilde' (pandas Series)
                   og 'denom' (pandas DataFrame).
      hp_coef    : Ordbog med koefficienter for hvert hp_year. Nøglerne er hp_year (som strings).
                   For hvert hp_year er der en ordbog med nøgler svarende til p (som strings),
                   der indeholder en liste med koefficienter for hver l-værdi.
      p_vec      : Liste af p-værdier (fx [0, 10, 20, ...]).
      l_vec      : Liste af regulariseringsparametre.
      hp_years   : Liste af hp-år (fx [2016, 2017, ...]).  
      orig_feat  : Bool – angiver om de originale features skal medtages.

    Returnerer:
      validation : Et pandas DataFrame med realiserede nytte-værdier samt tilhørende parametre.
    """
    # Konverter nøglerne for pfml_input["reals"] til pd.Timestamp og opret en mapping
    reals_all_raw = pfml_input["reals"]
    reals_all_dict = {pd.to_datetime(k): v for k, v in reals_all_raw.items()}

    # Liste til at samle alle rækker
    rows = []

    # For hvert hp_year (bruges som 'end')
    for end in sorted(hp_years):
        # Vælg observationer med datoer: >= (end-1)-12-31 og <= end-11-30
        lower_bound = pd.to_datetime(f"{end - 1}-12-31")
        upper_bound = pd.to_datetime(f"{end}-11-30")
        # Filtrer reals for dette hp_year
        reals_all = {dt: obj for dt, obj in reals_all_dict.items() if lower_bound <= dt <= upper_bound}
        
        # Hent koefficienterne for dette hp_year
        coef_list_yr = hp_coef[str(end)]
        
        # For hvert p i p_vec
        for p in p_vec:
            # Hent de relevante feature-navne for p
            feat_p = pfml_feat_fun(p, orig_feat)  # Forudsætter at funktionen er defineret
            # Hent koefficientlisten for denne p-værdi (nøglen som string)
            coef_list_p = coef_list_yr[str(p)]
            
            # For hver observation i reals_all: udtræk r_tilde og denom for feat_p
            reals_subset = {}
            for dt, obj in reals_all.items():
                # Restriktion af r_tilde og denom til de features, der er i feat_p
                r_tilde_sub = obj["r_tilde"][feat_p]
                denom_sub = obj["denom"].loc[feat_p, feat_p]
                reals_subset[dt] = {"r_tilde": r_tilde_sub, "denom": denom_sub}
            
            # For hver l-værdi (brug indeks i l_vec)
            for i, l_value in enumerate(l_vec):
                # Hent koefficienten for denne l-værdi
                coef = coef_list_p[i]  # Forventet at være en pandas Series med index = feat_p
                # For hver dato (nøgle) i reals_subset, beregn realiseret nytte
                for dt, sub_obj in reals_subset.items():
                    # Beregn: r = t(r_tilde) %*% coef - 0.5 * t(coef) %*% denom %*% coef
                    r_val = sub_obj["r_tilde"].dot(coef) - 0.5 * (coef.dot(sub_obj["denom"].dot(coef)))
                    # Beregn eom_ret: her beregnes som månedsslut for næste måned
                    eom_ret = dt + MonthEnd(1)
                    # Gem række med de relevante oplysninger
                    rows.append({
                        "eom": dt,
                        "eom_ret": eom_ret,
                        "obj": r_val,
                        "l": l_value,
                        "p": p,
                        "hp_end": end
                    })

    # Sammensæt resultaterne til et DataFrame
    validation = pd.DataFrame(rows)
    
    # Sorter efter p, l og eom_ret
    validation = validation.sort_values(by=["p", "l", "eom_ret"]).reset_index(drop=True)
    
    # Beregn kumulativ middelværdi af 'obj' for hver gruppe (p, l) i kronologisk rækkefølge af eom_ret.
    validation["cum_obj"] = validation.groupby(["p", "l"])["obj"].transform(lambda x: x.expanding().mean())
    
    # For hver eom_ret, beregn rangeringen af -cum_obj (dvs. højere cum_obj giver lavere ranknummer)
    validation["rank"] = validation.groupby("eom_ret")["cum_obj"].rank(ascending=False)
    
    return validation


In [66]:
import pandas as pd
from pandas.tseries.offsets import MonthEnd, DateOffset

def pfml_aims_fun(pfml_input, validation, data_tc, hp_coef, hp_years, dates_oos, l_vec, orig_feat):
    """
    Opretter den optimale AIM-portefølje (for et g) baseret på de fundne hyperparametre.
    
    Parametre:
      pfml_input : Dictionary med nøglen "signal_t". Her skal pfml_input["signal_t"] være en dictionary,
                   hvor nøglerne er dato-strenge (fx "2020-01-31") og værdier er DataFrames med signaler.
      validation : Pandas DataFrame med bl.a. kolonnerne 'eom_ret', 'rank', 'l' og 'p'.
                   Her antages 'eom_ret' at være datetime, så vi kan udlede året.
      data_tc    : Pandas DataFrame med portefølje-data, indeholdende mindst 'id', 'eom' og 'valid'.
      hp_coef    : Dictionary med koefficienter for hvert out-of-sample år. Strukturen er:
                   hp_coef[str(oos_year)][str(p)][l_index] = koefficient (pandas Series).
      hp_years   : Liste med hp-år (fx [2016, 2017, ...]).
      dates_oos  : Liste af out-of-sample datoer (pd.Timestamp objekter).
      l_vec      : Liste med lambda-værdier.
      orig_feat  : Boolean, angivende om de originale features skal medtages.
      
    Returnerer:
      aim_pfs_list : Dictionary med nøgler svarende til hver dato i dates_oos, hvor hver værdi er en dictionary med nøglerne:
                     "aim_pf" : DataFrame med kolonnerne 'id', 'eom' og 'w_aim'.
                     "coef"   : Den valgte koefficient (pandas Series) for den aktuelle dato.
    """
    # Udvælg optimum hyperparametre: de rækker hvor eom_ret er i december og rank==1,
    # og udled hp_end som året fra eom_ret.
    validation = validation.copy()
    validation['hp_end'] = validation['eom_ret'].dt.year
    opt_hps = validation[(validation['eom_ret'].dt.month == 12) & (validation['rank'] == 1)][['hp_end', 'l', 'p']]
    
    print("Optimum hyperparametre (opt_hps):")
    print(opt_hps)
    
    aim_pfs_list = {}
    
    for d in dates_oos:
        # Udregn d_ret svarende til: d + 1 + måneder(1) - 1 
        # (dvs. at få månedsslut for den næste måned)
        d_ret = d + pd.Timedelta(days=1) + DateOffset(months=1) - pd.Timedelta(days=1)
        print("d_ret:")
        print(d_ret)
        
        oos_year = d_ret.year
        print("oos_year:")
        print(oos_year)
        
        # Vælg hp_year: hvis (oos_year - 1) findes blandt opt_hps.hp_end, så brug det,
        # ellers bruges den mindste værdi af opt_hps.hp_end.
        if (oos_year - 1) in opt_hps['hp_end'].values:
            hp_year = oos_year - 1
        else:
            hp_year = opt_hps['hp_end'].min()
        print("hp_year:")
        print(hp_year)
        
        # Vælg de hyperparametre, der matcher hp_year
        hps_d = opt_hps[opt_hps['hp_end'] == hp_year]
        print("hps_d:")
        print(hps_d)
        
        if hps_d.empty:
            print(f"Ingen hyperparameter fundet for hp_year: {hp_year} til dato: {d_ret}")
            continue
        else:
            print(f"Hyperparameter for hp_year {hp_year}:")
            print(hps_d)
        
        # Antag at der kun er én række – ellers kan man tage den første
        p_val = hps_d['p'].iloc[0]
        l_val = hps_d['l'].iloc[0]
        # Hent de relevante features via pfml_feat_fun (skal være defineret)
        feat = pfml_feat_fun(p=p_val, orig_feat=orig_feat)
        
        # Hent signalerne for dato d. Vi antager, at nøglerne i pfml_input["signal_t"] er i ISO-format.
        key_d = d.strftime('%Y-%m-%d')
        s = pfml_input["signal_t"][key_d][feat]
        
        # Find indeks for l_val i l_vec (brug evt. en tilnærmelsesmetode for floating point)
        try:
            l_no = l_vec.index(l_val)
        except ValueError:
            l_no = min(range(len(l_vec)), key=lambda i: abs(l_vec[i] - l_val))
        
        # Hent den korrekte koefficient
        key_oos_year = str(oos_year)
        key_p = str(p_val)
        coef = hp_coef[key_oos_year][key_p][l_no]
        
        # Filtrer data_tc for rækker, hvor valid==True og eom == d
        aim_pf_data = data_tc[(data_tc['valid'] == True) & (data_tc['eom'] == d)].copy()
        # Beregn w_aim = s %*% coef. Her antages det, at s er en DataFrame med samme rækkefølge som
        # de relevante rækker i data_tc, og at coef er en pandas Series med index svarende til kolonnerne i s.
        aim_pf_data['w_aim'] = s.dot(coef)
        aim_pf = aim_pf_data[['id', 'eom', 'w_aim']]
        
        aim_pfs_list[d] = {"aim_pf": aim_pf, "coef": coef}
    
    return aim_pfs_list


In [ ]:
    data_tc=chars,
    cov_list=barra_cov,
    lambda_list=lambda_list,
    risk_free=risk_free,
    features=features,
    wealth=wealth,
    mu=pf_set['mu'],
    gamma_rel=pf_set['gamma_rel'],
    dates_full=dates_m2,
    lb=pf_set['lb_hor'],
    scale=settings['pf_ml']['scale'],
    g_vec=settings['pf_ml']['g_vec'],
    p_vec=settings['pf_ml']['p_vec'],
    l_vec=settings['pf_ml']['l_vec'],
    orig_feat=settings['pf_ml']['orig_feat'],
    iter=10,
    rff_feat=True,
    seed=settings['seed_no']

In [82]:
print("data_tc:", type(data_tc))
print("cov_list:", type(cov_list))
print("lambda_list:", type(lambda_list))
print("features:", type(features))
print("risk_free:", type(risk_free))
print("wealth:", type(wealth))
print("mu:", type(pf_set['mu']))
print("gamma_rel:", type(pf_set['gamma_rel']))
print("dates_full:", type(dates_full))
print("dates_oos:", type(dates_oos))
print("lb:", type(pf_set['lb_hor']))
print("hp_years:", type(hp_years))
print("rff_feat:", type(rff_feat))
print("g_vec:", type(settings['pf_ml']['g_vec']))
print("p_vec:", type(settings['pf_ml']['p_vec']))
print("l_vec:", type(settings['pf_ml']['l_vec']))
print("scale:", type(settings['pf_ml']['scale']))
print("orig_feat:", type(settings['pf_ml']['orig_feat']))
print("iter:", type(iter))
print("hps:", type(hps))
print("seed:", type(settings['seed_no']))


(DatetimeIndex(['2018-11-30', '2018-12-31', '2019-01-31', '2019-02-28',
                '2019-03-31', '2019-04-30', '2019-05-31', '2019-06-30',
                '2019-07-31', '2019-08-31', '2019-09-30', '2019-10-31',
                '2019-11-30', '2019-12-31', '2020-01-31', '2020-02-29',
                '2020-03-31', '2020-04-30', '2020-05-31', '2020-06-30',
                '2020-07-31', '2020-08-31', '2020-09-30', '2020-10-31',
                '2020-11-30', '2020-12-31', '2021-01-31', '2021-02-28',
                '2021-03-31', '2021-04-30', '2021-05-31', '2021-06-30',
                '2021-07-31', '2021-08-31', '2021-09-30', '2021-10-31',
                '2021-11-30', '2021-12-31', '2022-01-31', '2022-02-28',
                '2022-03-31', '2022-04-30', '2022-05-31', '2022-06-30',
                '2022-07-31', '2022-08-31', '2022-09-30', '2022-10-31',
                '2022-11-30', '2022-12-31', '2023-01-31', '2023-02-28',
                '2023-03-31', '2023-04-30', '2023-05-31', '2023-

In [107]:
if isinstance(dates, tuple):
    if len(dates) == 1 and isinstance(dates[0], pd.DatetimeIndex):
        dates = list(dates[0])
    else:
        dates = list(dates)
if isinstance(lb, tuple):
    lb = lb[0]

In [129]:
type(chars)

pandas.core.frame.DataFrame

In [117]:
min_date = min(dates)
max_date = max(dates)

# Beregn startdatoen svarende til: min(dates) + 1 dag - (lb+1) måneder
start_date = (min_date + pd.Timedelta(days=1)) - pd.DateOffset(months=lb+1)

# Opret en række datoer: fra start_date til max(dates)+1 dag med månedlig frekvens (brug 'MS' for månedstart)
# og træk derefter 1 dag fra hver dato for at få månedsslutdatoer
dates_lb = pd.date_range(start=start_date, end=(max_date + pd.Timedelta(days=1)), freq='MS') - pd.Timedelta(days=1)


In [67]:
p_vec

[64, 128, 256, 512]

In [139]:
g_vec=settings['pf_ml']['g_vec'],
p_vec=settings['pf_ml']['p_vec'],

In [147]:
    min_date = dates[0].min().date()
    max_date = dates[0].max().date()
    start_date = (min_date + pd.Timedelta(days=1)) - pd.DateOffset(months=lb+1)
    # Her benyttes månedsslutdatoer som lookback-datoer
    dates_lb = pd.date_range(start=start_date, end=max_date, freq='M')
    if isinstance(g_vec, tuple):
        g_vec = list(g_vec)
    if isinstance(p_vec, tuple):
        p_vec = list(p_vec)
    # --- Oprettelse af Random Fourier Features ---
    if rff_feat:
        np.random.seed(seed)
        # Vælg feature-kolonner fra data_tc
        if isinstance(features, tuple) and len(features) == 1:
            features = features[0]
        X_features = data_tc[features].values
        rff_x = rff(X_features, p=p_max, g=g)
        rff_w = rff_x['W']
        X_cos = rff_x['X_cos']
        X_sin = rff_x['X_sin']
        # Kombiner cos- og sin-komponenterne
        rff_features = np.hstack([X_cos, X_sin])
        num = p_max // 2
        rff_colnames = [f"rff{i}_cos" for i in range(1, num+1)] + [f"rff{i}_sin" for i in range(1, num+1)]
        rff_df = pd.DataFrame(rff_features, columns=rff_colnames, index=data_tc.index)
        # Sammensæt data med de basis kolonner og de nye rff-features
        data = pd.concat([data_tc[['id', 'eom', 'valid', 'ret_ld1', 'tr_ld0', 'mu_ld0']].reset_index(drop=True),
                          rff_df.reset_index(drop=True)], axis=1)
        feat_new = list(rff_df.columns)
        if add_orig:
            data = pd.concat([data, data_tc[features].reset_index(drop=True)], axis=1)
            feat_new = feat_new + features
    else:
        cols = ['id', 'eom', 'valid', 'ret_ld1', 'tr_ld0', 'mu_ld0'] + features
        data = data_tc[cols].copy()
        feat_new = features.copy()
    
    feat_cons = feat_new + ['constant']

TypeError: 'Timestamp' object is not callable

# Implement funktionen

In [20]:
def create_lambda(x, ids):
    return np.diag(x[ids])

In [72]:
import time
import pandas as pd
import numpy as np

def pfml_implement(data_tc, cov_list, lambda_list, risk_free, features, 
                   wealth, mu, gamma_rel, dates_full, dates_oos, lb, 
                   hp_years, rff_feat, scale, 
                   g_vec=None, p_vec=None, l_vec=None, orig_feat=None, iter=None, hps=None, balanced=None, seed=None):
    # Hyperparameter search
    if hps is None:
        start_time = time.time()
        hps = {}
        for g in g_vec:
            start_g = time.time()
            print(f"Processing g: {g}")
            # Step 1: Prepare data using pfml_input_fun
            pfml_input = pfml_input_fun(
                data_tc, wealth=wealth, mu=mu, dates=dates_full,
                lb=lb, scale=scale, risk_free=risk_free,
                cov_list=cov_list, lambda_list=lambda_list, gamma_rel=gamma_rel, iter=iter,
                features=features, balanced=balanced,
                rff_feat=rff_feat, p_max=max(p_vec), g=g, add_orig=orig_feat, seed=seed
            )
            print(f"pfml_input_fun took {time.time()-start_g:.2f} sec")
            
            # RFF weights
            rff_w = pfml_input['rff_w']
            # Inputs: get all desired features via pfml_feat_fun
            feat_all = pfml_feat_fun(p=max(p_vec), orig_feat=orig_feat)
            # Adjust pfml_input so that only the columns corresponding to feat_all remain.
            #pfml_input = adjust_pfml_input(pfml_input, feat_all)
            pfml_input = {
                "reals": [
                    {
                        "r_tilde": item["r_tilde"][feat_all],
                        **{key: value[np.ix_(feat_all, feat_all)] for key, value in item.items() if key != "r_tilde"}
                    }
                    for item in pfml_input["reals"]
                ],
                "signal_t": [matrix[:, feat_all] for matrix in pfml_input["signal_t"]]
            }
            # Coefficients
            pfml_hp_coef = pfml_search_coef(pfml_input, p_vec=p_vec, l_vec=l_vec, hp_years=hp_years, orig_feat=orig_feat)
            # Realized utility (validation)
            validation = pfml_hp_reals_fun(pfml_input, hp_coef=pfml_hp_coef, p_vec=p_vec, l_vec=l_vec, hp_years=hp_years, orig_feat=orig_feat)
            validation['g'] = g  # Add column 'g'
            
            # Find optimal aim portfolio
            aims = pfml_aims_fun(pfml_input, validation=validation, data_tc=data_tc, hp_coef=pfml_hp_coef, 
                                  hp_years=hp_years, dates_oos=dates_oos, l_vec=l_vec, orig_feat=orig_feat)
            elapsed_g = (time.time()-start_g)/60
            print(f"g: {g:.2f} took {elapsed_g:.2f} min")
            hps[str(g)] = {"aim_pfs_list": aims, "validation": validation, "rff_w": rff_w}
        total_elapsed = time.time()-start_time
        print(f"Total hyperparameter search took {total_elapsed:.2f} sec")
    
    print("hps after search:")
    print(hps)
    
    # Implement final portfolio
    # Combine all validation DataFrames from hps into one DataFrame.
    validation_list = [hps[g_key]['validation'] for g_key in hps]
    best_hps = pd.concat(validation_list, ignore_index=True)
    if 'rank' in best_hps.columns:
        best_hps = best_hps.drop(columns='rank')
    best_hps['rank'] = best_hps.groupby('eom_ret')['cum_obj'].rank(ascending=False, method='min')
    best_hps['eom_ret'] = pd.to_datetime(best_hps['eom_ret'])
    best_hps = best_hps[(best_hps['rank'] == 1) & (best_hps['eom_ret'].dt.month == 12)]
    
    # (Optional: Create a diagnostic plot here – omitted or commented out)
    # For example, using seaborn:
    # import seaborn as sns
    # import matplotlib.pyplot as plt
    # melt_df = best_hps.melt(id_vars=['eom_ret'], value_vars=['p', 'l', 'g'], var_name='param', value_name='value')
    # plt.figure(figsize=(10,6))
    # sns.scatterplot(data=melt_df, x='eom_ret', y='value', hue='param', alpha=0.5)
    # sns.lineplot(data=melt_df, x='eom_ret', y='value', hue='param')
    # plt.title("HP Range Check")
    # plt.show()
    
    # Aim portfolios: for each date in dates_oos
    best_hps_list = {}
    for d in dates_oos:
        d_ret = d + pd.offsets.MonthEnd(1)  # last day of next month
        oos_year = d_ret.year
        print("Processing aim portfolio for date:", d)
        # Filter best_hps for rows with hp_end equal to oos_year - 1
        best_hps_d = best_hps[best_hps['hp_end'] == (oos_year - 1)]
        if best_hps_d.empty:
            print(f"No best_hps found for year {oos_year-1}")
            continue
        best_row = best_hps_d.iloc[0]
        best_g = best_row['g']
        best_p = best_row['p']
        d_str = d.strftime('%Y-%m-%d')
        print("d_str",d_str)
        if str(best_g) not in hps or d_str not in hps[str(best_g)]["aim_pfs_list"]:
            print(f"No aim portfolio found for date {d_str} under best_g {best_g}")
            continue
        best_aim = hps[str(best_g)]["aim_pfs_list"][d_str]["aim_pf"]
        best_coef = hps[str(best_g)]["aim_pfs_list"][d_str]["coef"]
        best_hps_list[d_str] = {"g": best_g, "p": best_p, "aim": best_aim, "coef": best_coef}
    
    if len(best_hps_list) == 0:
        raise ValueError("No aim portfolios were generated!")
    
    aims_df = pd.concat([entry["aim"] for entry in best_hps_list.values()], ignore_index=True)
    
    # Final portfolio: select data_tc for dates in dates_oos and valid == True, then call pfml_w.
    w_data = data_tc[(data_tc['valid'] == True) & (data_tc['eom'].isin(dates_oos))]
    w_data = w_data[['id', 'eom', 'eom_ret', 'me', 'tr_ld1', 'valid']]
    w = pfml_w(w_data, dates=dates_oos, cov_list=cov_list, lambda_list=lambda_list, gamma_rel=gamma_rel, iter=iter, risk_free=risk_free, wealth=wealth, mu=mu, aims=aims_df)
    pf = pf_ts_fun(w, data=data_tc, wealth=wealth, gam=gamma_rel)
    pf['type'] = "Portfolio-ML"
    rff_w_list = {g: hps[str(g)]["rff_w"] for g in g_vec}
    return {"hps": hps, "best_hps": best_hps, "best_hps_list": best_hps_list, "aims": aims_df, "w": w, "pf": pf, "rff_w_list": rff_w_list}


In [73]:
import warnings
warnings.filterwarnings("ignore")

pfml = pfml_implement(
    data_tc=chars, #rigtig
    cov_list=barra_cov, #rigtig
    lambda_list=lambda_list, #rigtig
    features=features, #rigtig
    risk_free=risk_free, # forkert
    wealth=wealth, # forkert
    mu=pf_set['mu'], #rigtig
    gamma_rel=pf_set['gamma_rel'], #rigtig
    dates_full=dates_m2,
    dates_oos=dates_oos,  # forkert
    lb=pf_set['lb_hor'], # rigtig
    hp_years=hp_years, # forkert
    rff_feat=True, #rigtig
    g_vec=settings['pf_ml']['g_vec'], # rigtig
    p_vec=settings['pf_ml']['p_vec'], # rigtig
    l_vec=settings['pf_ml']['l_vec'], # rigtig
    scale=settings['pf_ml']['scale'], # rigtig
    orig_feat=settings['pf_ml']['orig_feat'], #rigtig
    iter=10,                # rigtig
    hps=None,          #rigtig
    balanced=False,
    seed=settings['seed_no'] #rigtig
)


Processing g: 0.049787068367863944
--> PF-ML inputs: 2020-01-31 00:00:00
d 2020-01-31 00:00:00
d_str 2020-01-31
m_func: n = 172
Advarsel: Ingen signal fundet for lookback-dato 2019-01-31
d 2020-02-29 00:00:00
d_str 2020-02-29
m_func: n = 172
Advarsel: Ingen signal fundet for lookback-dato 2019-02-28
d 2020-03-31 00:00:00
d_str 2020-03-31
m_func: n = 172
Advarsel: Ingen signal fundet for lookback-dato 2019-03-31
d 2020-04-30 00:00:00
d_str 2020-04-30
m_func: n = 172
Advarsel: Ingen signal fundet for lookback-dato 2019-04-30
d 2020-05-31 00:00:00
d_str 2020-05-31
m_func: n = 172
Advarsel: Ingen signal fundet for lookback-dato 2019-05-31
d 2020-06-30 00:00:00
d_str 2020-06-30
m_func: n = 172
Advarsel: Ingen signal fundet for lookback-dato 2019-06-30
d 2020-07-31 00:00:00
d_str 2020-07-31
m_func: n = 172
Advarsel: Ingen signal fundet for lookback-dato 2019-07-31
d 2020-08-31 00:00:00
d_str 2020-08-31
m_func: n = 172
Advarsel: Ingen signal fundet for lookback-dato 2019-08-31
d 2020-09-30 00

TypeError: string indices must be integers, not 'str'

## Test funktioner

In [163]:
import copy

data_tc = copy.deepcopy(chars)
cov_list = copy.deepcopy(barra_cov)
lambda_list = copy.deepcopy(lambda_list)
risk_free = copy.deepcopy(risk_free)
features = copy.deepcopy(features)
wealth = copy.deepcopy(wealth)
mu = copy.deepcopy(pf_set['mu'])
gamma_rel = copy.deepcopy(pf_set['gamma_rel'])
dates_full = copy.deepcopy(dates_m2)
lb = copy.deepcopy(pf_set['lb_hor'])
scale = copy.deepcopy(settings['pf_ml']['scale'])
g_vec = copy.deepcopy(settings['pf_ml']['g_vec'])
p_vec = copy.deepcopy(settings['pf_ml']['p_vec'])
l_vec = copy.deepcopy(settings['pf_ml']['l_vec'])
orig_feat = copy.deepcopy(settings['pf_ml']['orig_feat'])
iter = copy.deepcopy(10)
rff_feat = copy.deepcopy(True)
seed = copy.deepcopy(settings['seed_no'])


In [165]:
print("data_tc:", type(data_tc))
print("cov_list:", type(cov_list))
print("lambda_list:", type(lambda_list))
print("features:", type(features))
print("risk_free:", type(risk_free))
print("wealth:", type(wealth))
print("mu:", type(pf_set['mu']))
print("gamma_rel:", type(pf_set['gamma_rel']))
print("dates_full:", type(dates_full))
print("dates_oos:", type(dates_oos))
print("lb:", type(pf_set['lb_hor']))
print("hp_years:", type(hp_years))
print("rff_feat:", type(rff_feat))
print("g_vec:", type(settings['pf_ml']['g_vec']))
print("p_vec:", type(settings['pf_ml']['p_vec']))
print("l_vec:", type(settings['pf_ml']['l_vec']))
print("scale:", type(settings['pf_ml']['scale']))
print("orig_feat:", type(settings['pf_ml']['orig_feat']))
print("iter:", type(iter))
print("hps:", type(hps))
print("seed:", type(settings['seed_no']))


data_tc: <class 'tuple'>
cov_list: <class 'tuple'>
lambda_list: <class 'tuple'>
features: <class 'tuple'>
risk_free: <class 'tuple'>
wealth: <class 'tuple'>
mu: <class 'float'>
gamma_rel: <class 'int'>
dates_full: <class 'tuple'>
dates_oos: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>
lb: <class 'int'>
hp_years: <class 'list'>
rff_feat: <class 'tuple'>
g_vec: <class 'list'>
p_vec: <class 'list'>
l_vec: <class 'list'>
scale: <class 'bool'>
orig_feat: <class 'bool'>
iter: <class 'tuple'>
hps: <class 'dict'>
seed: <class 'int'>


In [69]:
pfml_input_fun(
            data_tc, wealth=wealth, mu=mu, dates=dates_full,
            lb=lb, scale=scale, risk_free=risk_free,
            cov_list=cov_list, lambda_list=lambda_list, gamma_rel=gamma_rel, iter=iter,
            features=features, balanced=False,
            rff_feat=rff_feat, p_max=max(p_vec), g=g, add_orig=orig_feat, seed=seed
        )

NameError: name 'g' is not defined

In [ ]:
def pfml_hyperparameter_search(data_tc, cov_list, lambda_list, risk_free, features, 
                               wealth, mu, gamma_rel, dates_full, lb, scale,
                               g_vec, p_vec, l_vec, orig_feat, iter, rff_feat, seed):
    # Konverter 'dates' til en liste, hvis det er en tuple
    if isinstance(dates_full, tuple):
        if len(dates_full) == 1 and isinstance(dates_full[0], pd.DatetimeIndex):
            dates_full = list(dates_full[0])
        else:
            dates_full = list(dates_full)
    if isinstance(g_vec, tuple):
        g_vec = list(g_vec)
    if isinstance(p_vec, tuple):
        p_vec = list(p_vec)
    if isinstance(lb, tuple):
        lb = lb[0]
    hps = {}
    for g in g_vec:
        start_g = time.time()
        print(f"Processing g: {g}")
        pfml_input = pfml_input_fun(
            data_tc, wealth=wealth, mu=mu, dates=dates_full,
            lb=lb, scale=scale, risk_free=risk_free,
            cov_list=cov_list, lambda_list=lambda_list, gamma_rel=gamma_rel, iter=iter,
            features=features, balanced=False,
            rff_feat=rff_feat, p_max=max(p_vec), g=g, add_orig=orig_feat, seed=seed
        )
        print(f"pfml_input_fun took {time.time()-start_g:.2f} sec")
        
        rff_w = pfml_input['rff_w']
        feat_all = pfml_feat_fun(p=max(p_vec), orig_feat=orig_feat)
        pfml_input = adjust_pfml_input(pfml_input, feat_all)
        pfml_hp_coef = pfml_search_coef(pfml_input, p_vec=p_vec, l_vec=l_vec, hp_years=hp_years, orig_feat=orig_feat)
        validation = pfml_hp_reals_fun(pfml_input, hp_coef=pfml_hp_coef, p_vec=p_vec, l_vec=l_vec, hp_years=hp_years, orig_feat=orig_feat)
        validation['g'] = g
        aims = pfml_aims_fun(pfml_input, validation=validation, data_tc=data_tc, hp_coef=pfml_hp_coef, 
                              hp_years=hp_years, dates_oos=dates_oos, l_vec=l_vec, orig_feat=orig_feat)
        elapsed_g = (time.time()-start_g)/60
        print(f"g: {g:.2f} took {elapsed_g:.2f} min")
        hps[str(g)] = {"aim_pfs_list": aims, "validation": validation, "rff_w": rff_w}
    return hps

In [151]:
hps = pfml_hyperparameter_search(
    data_tc=chars,
    cov_list=barra_cov,
    lambda_list=lambda_list,
    risk_free=risk_free,
    features=features,
    wealth=wealth,
    mu=pf_set['mu'],
    gamma_rel=pf_set['gamma_rel'],
    dates_full=dates_m2,
    lb=pf_set['lb_hor'],
    scale=settings['pf_ml']['scale'],
    g_vec=settings['pf_ml']['g_vec'],
    p_vec=settings['pf_ml']['p_vec'],
    l_vec=settings['pf_ml']['l_vec'],
    orig_feat=settings['pf_ml']['orig_feat'],
    iter=10,
    rff_feat=True,
    seed=settings['seed_no']
)

Processing g: 0.049787068367863944


TypeError: 'Timestamp' object is not callable

In [149]:


def pfml_hyperparameter_search(data_tc, cov_list, lambda_list, risk_free, features, 
                               wealth, mu, gamma_rel, dates_full, lb, scale,
                               g_vec, p_vec, l_vec, orig_feat, iter, rff_feat, seed):
    # Konverter 'dates' til en liste, hvis det er en tuple
    if isinstance(dates_full, tuple):
        if len(dates_full) == 1 and isinstance(dates_full[0], pd.DatetimeIndex):
            dates_full = list(dates_full[0])
        else:
            dates_full = list(dates_full)
    if isinstance(g_vec, tuple):
        g_vec = list(g_vec)
    if isinstance(p_vec, tuple):
        p_vec = list(p_vec)
    if isinstance(lb, tuple):
        lb = lb[0]
    hps = {}
    for g in g_vec:
        start_g = time.time()
        print(f"Processing g: {g}")
        pfml_input = pfml_input_fun(
            data_tc, wealth=wealth, mu=mu, dates=dates_full,
            lb=lb, scale=scale, risk_free=risk_free,
            cov_list=cov_list, lambda_list=lambda_list, gamma_rel=gamma_rel, iter=iter,
            features=features, balanced=False,
            rff_feat=rff_feat, p_max=max(p_vec), g=g, add_orig=orig_feat, seed=seed
        )
        print(f"pfml_input_fun took {time.time()-start_g:.2f} sec")
        
        rff_w = pfml_input['rff_w']
        feat_all = pfml_feat_fun(p=max(p_vec), orig_feat=orig_feat)
        pfml_input = adjust_pfml_input(pfml_input, feat_all)
        pfml_hp_coef = pfml_search_coef(pfml_input, p_vec=p_vec, l_vec=l_vec, hp_years=hp_years, orig_feat=orig_feat)
        validation = pfml_hp_reals_fun(pfml_input, hp_coef=pfml_hp_coef, p_vec=p_vec, l_vec=l_vec, hp_years=hp_years, orig_feat=orig_feat)
        validation['g'] = g
        aims = pfml_aims_fun(pfml_input, validation=validation, data_tc=data_tc, hp_coef=pfml_hp_coef, 
                              hp_years=hp_years, dates_oos=dates_oos, l_vec=l_vec, orig_feat=orig_feat)
        elapsed_g = (time.time()-start_g)/60
        print(f"g: {g:.2f} took {elapsed_g:.2f} min")
        hps[str(g)] = {"aim_pfs_list": aims, "validation": validation, "rff_w": rff_w}
    return hps

def select_aim_portfolios(hps, best_hps, dates_oos):
    best_hps_list = {}
    for d in dates_oos:
        d_ret = d + pd.offsets.MonthEnd(1)
        oos_year = d_ret.year
        print("Processing aim portfolio for date:", d)
        best_hps_d = best_hps[best_hps['hp_end'] == (oos_year - 1)]
        if best_hps_d.empty:
            print(f"No best_hps found for year {oos_year-1}")
            continue
        best_row = best_hps_d.iloc[0]
        best_g = best_row['g']
        best_p = best_row['p']
        d_str = d.strftime('%Y-%m-%d')
        if str(best_g) not in hps or d_str not in hps[str(best_g)]["aim_pfs_list"]:
            print(f"No aim portfolio found for date {d_str} under best_g {best_g}")
            continue
        best_aim = hps[str(best_g)]["aim_pfs_list"][d_str]["aim_pf"]
        best_coef = hps[str(best_g)]["aim_pfs_list"][d_str]["coef"]
        best_hps_list[d_str] = {"g": best_g, "p": best_p, "aim": best_aim, "coef": best_coef}
    if len(best_hps_list) == 0:
        raise ValueError("No aim portfolios were generated!")
    return best_hps_list

def compute_final_portfolio(data_tc, dates_oos, cov_list, lambda_list, gamma_rel, iter, 
                            risk_free, wealth, mu, best_hps_list):
    aims_df = pd.concat([entry["aim"] for entry in best_hps_list.values()], ignore_index=True)
    w_data = data_tc[(data_tc['valid'] == True) & (data_tc['eom'].isin(dates_oos))]
    w_data = w_data[['id', 'eom', 'eom_ret', 'me', 'tr_ld1', 'valid']]
    w = pfml_w(w_data, dates=dates_oos, cov_list=cov_list, lambda_list=lambda_list, gamma_rel=gamma_rel, 
               iter=iter, risk_free=risk_free, wealth=wealth, mu=mu, aims=aims_df)
    pf = pf_ts_fun(w, data=data_tc, wealth=wealth, gam=gamma_rel)
    pf['type'] = "Portfolio-ML"
    return w, pf

def pfml_implement(data_tc, cov_list, lambda_list, risk_free, features, 
                   wealth, mu, gamma_rel, dates_full, dates_oos, lb, 
                   hp_years, rff_feat, scale, 
                   g_vec=None, p_vec=None, l_vec=None, orig_feat=None, iter=None, hps=None, balanced=None, seed=None):
    # Hyperparameter search
    if hps is None:
        start_time = time.time()
        hps = pfml_hyperparameter_search(
            data_tc=data_tc, cov_list=cov_list, lambda_list=lambda_list, risk_free=risk_free, features=features,
            wealth=wealth, mu=mu, gamma_rel=gamma_rel, dates_full=dates_full, lb=lb, scale=scale,
            g_vec=g_vec, p_vec=p_vec, l_vec=l_vec, orig_feat=orig_feat, iter=iter, rff_feat=rff_feat, seed=seed
        )
        total_elapsed = time.time()-start_time
        print(f"Total hyperparameter search took {total_elapsed:.2f} sec")
    
    print("hps after search:")
    print(hps)
    
    # Combine validation DataFrames og udvælg best_hps
    validation_list = [hps[g_key]['validation'] for g_key in hps]
    best_hps = pd.concat(validation_list, ignore_index=True)
    if 'rank' in best_hps.columns:
        best_hps = best_hps.drop(columns='rank')
    best_hps['rank'] = best_hps.groupby('eom_ret')['cum_obj'].rank(ascending=False, method='min')
    best_hps['eom_ret'] = pd.to_datetime(best_hps['eom_ret'])
    best_hps = best_hps[(best_hps['rank'] == 1) & (best_hps['eom_ret'].dt.month == 12)]
    
    # Udvælg aim porteføljer
    best_hps_list = select_aim_portfolios(hps, best_hps, dates_oos)
    
    # Final portfolio beregning
    w, pf = compute_final_portfolio(data_tc, dates_oos, cov_list, lambda_list, gamma_rel, iter, risk_free, wealth, mu, best_hps_list)
    
    rff_w_list = {g: hps[str(g)]["rff_w"] for g in g_vec}
    return {"hps": hps, "best_hps": best_hps, "best_hps_list": best_hps_list, "aims": pd.concat([entry["aim"] for entry in best_hps_list.values()], ignore_index=True),
            "w": w, "pf": pf, "rff_w_list": rff_w_list}


In [150]:


hps = pfml_hyperparameter_search(
    data_tc=chars,
    cov_list=barra_cov,
    lambda_list=lambda_list,
    risk_free=risk_free,
    features=features,
    wealth=wealth,
    mu=pf_set['mu'],
    gamma_rel=pf_set['gamma_rel'],
    dates_full=dates_m2,
    lb=pf_set['lb_hor'],
    scale=settings['pf_ml']['scale'],
    g_vec=settings['pf_ml']['g_vec'],
    p_vec=settings['pf_ml']['p_vec'],
    l_vec=settings['pf_ml']['l_vec'],
    orig_feat=settings['pf_ml']['orig_feat'],
    iter=10,
    rff_feat=True,
    seed=settings['seed_no']
)


Processing g: 0.049787068367863944


TypeError: 'Timestamp' object is not callable

In [128]:
type(chars)

pandas.core.frame.DataFrame

## samlet

In [164]:
print("data_tc:", type(data_tc))
print("cov_list:", type(cov_list))
print("lambda_list:", type(lambda_list))
print("features:", type(features))
print("risk_free:", type(risk_free))
print("wealth:", type(wealth))
print("mu:", type(pf_set['mu']))
print("gamma_rel:", type(pf_set['gamma_rel']))
print("dates_full:", type(dates_full))
print("dates_oos:", type(dates_oos))
print("lb:", type(pf_set['lb_hor']))
print("hp_years:", type(hp_years))
print("rff_feat:", type(rff_feat))
print("g_vec:", type(settings['pf_ml']['g_vec']))
print("p_vec:", type(settings['pf_ml']['p_vec']))
print("l_vec:", type(settings['pf_ml']['l_vec']))
print("scale:", type(settings['pf_ml']['scale']))
print("orig_feat:", type(settings['pf_ml']['orig_feat']))
print("iter:", type(iter))
print("hps:", type(hps))
print("seed:", type(settings['seed_no']))


data_tc: <class 'tuple'>
cov_list: <class 'tuple'>
lambda_list: <class 'tuple'>
features: <class 'tuple'>
risk_free: <class 'tuple'>
wealth: <class 'tuple'>
mu: <class 'float'>
gamma_rel: <class 'int'>
dates_full: <class 'tuple'>
dates_oos: <class 'pandas.core.indexes.datetimes.DatetimeIndex'>
lb: <class 'int'>
hp_years: <class 'list'>
rff_feat: <class 'tuple'>
g_vec: <class 'list'>
p_vec: <class 'list'>
l_vec: <class 'list'>
scale: <class 'bool'>
orig_feat: <class 'bool'>
iter: <class 'tuple'>
hps: <class 'dict'>
seed: <class 'int'>


In [70]:
import warnings
warnings.filterwarnings("ignore")

pfml = pfml_implement(
    data_tc=chars, #rigtig
    cov_list=barra_cov, #rigtig
    lambda_list=lambda_list, #rigtig
    features=features, #rigtig
    risk_free=risk_free, # forkert
    wealth=wealth, # forkert
    mu=pf_set['mu'], #rigtig
    gamma_rel=pf_set['gamma_rel'], #rigtig
    dates_full=dates_m2,
    dates_oos=dates_oos,  # forkert
    lb=pf_set['lb_hor'], # rigtig
    hp_years=hp_years, # forkert
    rff_feat=True, #rigtig
    g_vec=settings['pf_ml']['g_vec'], # rigtig
    p_vec=settings['pf_ml']['p_vec'], # rigtig
    l_vec=settings['pf_ml']['l_vec'], # rigtig
    scale=settings['pf_ml']['scale'], # rigtig
    orig_feat=settings['pf_ml']['orig_feat'], #rigtig
    iter=10,                # rigtig
    hps=None,          #rigtig
    balanced=False,
    seed=settings['seed_no'] #rigtig
)


Processing g: 0.049787068367863944
--> PF-ML inputs: 2020-01-31 00:00:00
d 2020-01-31 00:00:00
d_str 2020-01-31
m_func: n = 172
Advarsel: Ingen signal fundet for lookback-dato 2019-01-31
d 2020-02-29 00:00:00
d_str 2020-02-29
m_func: n = 172
Advarsel: Ingen signal fundet for lookback-dato 2019-02-28
d 2020-03-31 00:00:00
d_str 2020-03-31
m_func: n = 172
Advarsel: Ingen signal fundet for lookback-dato 2019-03-31
d 2020-04-30 00:00:00
d_str 2020-04-30
m_func: n = 172
Advarsel: Ingen signal fundet for lookback-dato 2019-04-30
d 2020-05-31 00:00:00
d_str 2020-05-31
m_func: n = 172
Advarsel: Ingen signal fundet for lookback-dato 2019-05-31
d 2020-06-30 00:00:00
d_str 2020-06-30
m_func: n = 172
Advarsel: Ingen signal fundet for lookback-dato 2019-06-30
d 2020-07-31 00:00:00
d_str 2020-07-31
m_func: n = 172
Advarsel: Ingen signal fundet for lookback-dato 2019-07-31
d 2020-08-31 00:00:00
d_str 2020-08-31
m_func: n = 172
Advarsel: Ingen signal fundet for lookback-dato 2019-08-31
d 2020-09-30 00

NameError: name 'adjust_pfml_input' is not defined